# Card Generating DCGAN 

In [1]:
import json
import binascii
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
#import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader, TensorDataset

# Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
torch.use_deterministic_algorithms(True) # Needed for reproducible results

Random Seed:  999


What machine learning models are used for variable input and output sizes?

Recurrent Neural Networks (RNNs): RNNs are a class of neural networks that process sequential data one element at a time while maintaining an internal hidden state. They can handle variable-length sequences by iterating over the elements one by one.

Long Short-Term Memory Networks (LSTMs) and Gated Recurrent Units (GRUs): These are variants of RNNs designed to mitigate the vanishing gradient problem and better capture long-range dependencies in sequences.

Encoder-Decoder Models: These architectures consist of two main components: an encoder that processes the input sequence and a decoder that generates the output sequence. They are commonly used for sequence-to-sequence tasks like machine translation.

Transformer: The Transformer architecture, introduced in the "Attention is All You Need" paper, is based on self-attention mechanisms. It has become a fundamental model in natural language processing tasks and allows for parallel processing of input and output sequences, making it particularly efficient for variable-length sequences.

Attention-based Models: Attention mechanisms can be used in combination with other models (e.g., RNNs, LSTMs, or Transformers) to focus on specific parts of the input sequence when generating the output sequence. This helps the model effectively handle variable-sized inputs and outputs.

Convolutional Sequence-to-Sequence Models: Inspired by convolutional neural networks, these models use 1D convolutions to process sequential data, allowing them to handle variable-length input and output sequences.

Pointer Networks: Pointer networks can be used to generate sequences by selectively copying elements from the input sequence, which is especially useful when the output vocabulary is not fixed.

Graph Neural Networks (GNNs): GNNs can handle variable input sizes when dealing with graph-structured data, as they operate on the graph's nodes and edges, which can have varying degrees.

** Convolutional Sequence-to-Sequence Models might work **

In [2]:
class CustomImageDataset(Dataset):
    
    def __init__(self):
        # Opening JSON file as a dictionary
        with open('cards.collectible.json', encoding='utf-8') as f:
            data = json.load(f)
        # Create new list of strings
        cardList = []
        for i in data['allCards']:
            if (i["type"] == "HERO"):
                continue
            cardString = i['name'] 
            cardString += " " + i["cardClass"] + " " + i["type"]
            cardString += " " + str(i["cost"]) + " mana"
            if (i["type"] == "MINION"):
                cardString += " " + str(i["attack"]) + "\\" + str(i["health"])
            #cardString += " \"" + i["text"] + "\""
            cardList.append(cardString)
        # Closing file
        f.close()
        # Find the length of the largest string
        #maxStringLength = len(cardList[1])
        #for i in cardList:
        #    if len(i) > maxStringLength:
        #        maxStringLength = len(i)
        #print(maxStringLength) # as of 7/29/2023, it is 56
        # Make all strings equal size (64)
        for i in range(len(cardList)):
            while(len(cardList[i]) < 64):
                cardList[i] += " "
        # Create a list of tensors
        self.tensorList = []
        for i in range(len(cardList)):
            characterTensors = []
            for c in cardList[i]:
                binaryCharacter = bin(ord(c))
                #remove '0b' from beginning
                binaryCharacter = binaryCharacter[2:-1]
                binaryList = []
                for j in range(8):
                    if (j < len(binaryCharacter)):
                        binaryList.append(int(binaryCharacter[j]))
                    else:
                        binaryList.append(0)   
                characterTensors.append(binaryList)
            self.tensorList.append(torch.tensor(characterTensors))

    def __len__(self):
        return len(self.tensorList)

    def __getitem__(self, idx):
        return self.tensorList[idx], 1

In [3]:
#dataset = CustomImageDataset()
#print(dataset.__len__())
#print(len(dataset.__getitem__(0)[0]))
#print(dataset.__getitem__(0)[0].shape)
#print(dataset.__getitem__(0))

In [4]:
# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 1

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 5

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparameter for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

In [5]:
# Opening JSON file as a dictionary
with open('cards.collectible.json', encoding='utf-8') as f:
    data = json.load(f)

# Create new list of strings
cardList = []
for i in data['allCards']:
    if (i["type"] == "HERO"):
        continue
    cardString = i['name'] 
    cardString += " " + i["cardClass"] + " " + i["type"]
    cardString += " " + str(i["cost"]) + " mana"
    if (i["type"] == "MINION"):
        cardString += " " + str(i["attack"]) + "\\" + str(i["health"])
    #cardString += " \"" + i["text"] + "\""
    cardList.append(cardString)
# Closing file
f.close()
# Make all strings equal size (64)
for i in range(len(cardList)):
    while(len(cardList[i]) < 64):
        cardList[i] += " "

# Create a list of tensors
tensorList = []
for i in range(len(cardList)):
    characterTensors = []
    for c in cardList[i]:
        binaryCharacter = bin(ord(c))
        #remove '0b' from beginning
        binaryCharacter = binaryCharacter[2:-1]
        binaryList = []
        for j in range(8):
            if (j < len(binaryCharacter)):
                binaryList.append(int(binaryCharacter[j]))
            else:
                binaryList.append(0)   
        characterTensors.append(binaryList)
    tensorList.append([characterTensors])

data = torch.tensor(tensorList, dtype=torch.float32)
print(data.shape)
print(data.dtype)
labels = torch.ones(len(cardList)) #.reshape([1, len(cardList)])

# Create a TensorDataset and DataLoader
dataset = TensorDataset(data, labels)

torch.Size([5084, 1, 64, 8])
torch.float32


In [6]:
# We get the dataset from above
# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)#,
                                         #shuffle=True, num_workers=workers)

# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [7]:
# custom weights initialization called on ``netG`` and ``netD``
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [18]:
# Generator Code

#Input noise has size [128, 100, 1, 1]
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. ``(ngf*8) x 4 x 4``
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. ``(ngf*4) x 8 x 8``
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. ``(ngf*2) x 16 x 16``
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. ``(ngf) x 32 x 32``
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            # state size. ``(nc) x 64 x 64``
            nn.Conv2d(nc, nc, 4, (1, 8), (2, 0), bias=False),
            nn.Tanh()
            # state size. ``(nc) x 64 x 8``
        )

    def forward(self, input):
        output = self.main(input)
        binary_output = torch.where(output >= 0.5, torch.ones_like(output), torch.zeros_like(output))
        return binary_output

In [21]:
# Create the generator
netG = Generator(ngpu).to(device)

# Get an output of the generator
b_size = real_cpu.size(0)
noise = torch.randn(b_size, nz, 1, 1, device=device)
fake = netG(noise)
print("fake is ", fake)

fake is  tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 1.,  ..., 0., 0., 1.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 1., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 1.,  ..., 0., 1., 1.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 1., 0., 1.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 1., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,

In [22]:
# Create the generator
netG = Generator(ngpu).to(device)

# Handle multi-GPU if desired
if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))

# Apply the ``weights_init`` function to randomly initialize all weights
#  to ``mean=0``, ``stdev=0.02``.
netG.apply(weights_init)

# Print the model
print(netG)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Conv2d(1, 1, 

In [23]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # torch.nn.Conv2d(in_channels, out_channels, 
            # kernel_size, stride=1, padding=0, dilation=1, groups=1, 
            # bias=True, padding_mode='zeros', device=None, dtype=None)
            # input is ``(nc) x 64 x 8``
            nn.Conv2d(nc, ndf, (4, 1), (2, 1), (1, 0), bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf) x 32 x 8``
            nn.Conv2d(ndf, ndf * 2, (4, 1), (2, 1), (1, 0), bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf*2) x 16 x 8``
            nn.Conv2d(ndf * 2, ndf * 4, (4, 1), (2, 1), (1, 0), bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf*4) x 8 x 8``
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf*8) x 4 x 4``
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
        #Warning: Lack of communication between columns (7/30/2023)

    def forward(self, input):
        return self.main(input)

In [24]:
# Create the Discriminator
netD = Discriminator(ngpu).to(device)

# Handle multi-GPU if desired
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))

# Apply the ``weights_init`` function to randomly initialize all weights
# like this: ``to mean=0, stdev=0.2``.
netD.apply(weights_init)

# Print the model
print(netD)

Discriminator(
  (main): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 1), stride=(2, 1), padding=(1, 0), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 1), stride=(2, 1), padding=(1, 0), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 1), stride=(2, 1), padding=(1, 0), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)


In [25]:
# Initialize the ``BCELoss`` function
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [26]:
#for i, data in enumerate(dataloader, start=0):
#    print(i)
#    print(data)

In [41]:
# Training Loop

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(dataloader, 0):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        netD.zero_grad()
        # Format batch
        real_cpu = data[0].to(device)
        #print("data shape is", real_cpu.shape)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        # Forward pass real batch through D
        output = netD(real_cpu).view(-1)
        #print("output shape is", output.shape)
        # Calculate loss on all-real batch
        errD_real = criterion(output, label)
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        #noise = torch.randn(b_size, nz, 1, 1, device=device)
        noise = torch.randint(2, (b_size, nz, 1, 1), dtype=torch.float, device=device)
        #print("noise shape is", noise.shape)
        # Generate fake image batch with G
        fake = netG(noise)
        #print("fake shape is", fake.shape)
        label.fill_(fake_label)
        # Classify all fake batch with D
        output = netD(fake.detach()).view(-1)
        #print("output shape is", output.shape)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, label)
        # Calculate the gradients for this batch, accumulated (summed) with previous gradients
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Compute error of D as sum over the fake and the real batches
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        # Calculate G's loss based on this output
        errG = criterion(output, label)
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()

        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1

Starting Training Loop...
[0/5][0/40]	Loss_D: 0.0000	Loss_G: 12.5086	D(x): 1.0000	D(G(z)): 0.0000 / 0.0000
[1/5][0/40]	Loss_D: 0.0000	Loss_G: 12.7214	D(x): 1.0000	D(G(z)): 0.0000 / 0.0000
[2/5][0/40]	Loss_D: 0.0000	Loss_G: 12.9035	D(x): 1.0000	D(G(z)): 0.0000 / 0.0000
[3/5][0/40]	Loss_D: 0.0000	Loss_G: 13.0654	D(x): 1.0000	D(G(z)): 0.0000 / 0.0000
[4/5][0/40]	Loss_D: 0.0000	Loss_G: 13.2150	D(x): 1.0000	D(G(z)): 0.0000 / 0.0000


In [46]:
noise = torch.randn(b_size, nz, 1, 1, device=device)
print(noise.shape)
noise = torch.where(noise >= 0, torch.ones_like(output), torch.zeros_like(output))
print(noise.shape)
noise = torch.randint(2, (b_size, nz, 1, 1), dtype=torch.float, device=device)
print(noise.shape)
fake = netG(noise)
print(fake)

torch.Size([92, 100, 1, 1])
torch.Size([92, 100, 1, 92])
torch.Size([92, 100, 1, 1])
tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
   

In [ ]:
dataiter = iter(dataloader)
single_data = next(dataiter)
print(single_data[0].shape)

In [ ]:
def read(t):
    